# Module 2. GluonTS Training on Amazon SageMaker
---

본 모듈에서는 Amazon SageMaker API를 호출하여 모델 훈련을 수행합니다. 노트북 실행에는 약 10분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

Amazon SageMaker는 완전관리형 머신 러닝 서비스로 인프라 관리에 대해 걱정할 필요가 없으며, 딥러닝 프레임워크의 훈련/배포 컨테이너 이미지를 가져 와서
여러분의 스크립트 코드를 쉽게 통합할 수 있습니다.

<br>

## 1. Training script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 훈련 스크립트인 `train.py`를 저장합니다.
아래 스크립트가 이전 모듈의 코드와 대부분 일치하다는 점을 알 수 있습니다. 다시 말해, SageMaker 훈련 스크립트 파일은 기존 온프레미스에서 사용했던 Python 스크립트 파일과 크게 다르지 않으며, SageMaker 훈련 컨테이너에서 수행하기 위한 추가적인 환경 변수들만 설정하시면 됩니다.

환경 변수 설정의 code snippet은 아래과 같습니다.

```python
# SageMaker Container environment
parser.add_argument('--model_dir', type=str, default=os.environ['SM_MODEL_DIR'])
parser.add_argument('--data_dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
parser.add_argument('--num_gpus', type=int, default=os.environ['SM_NUM_GPUS'])
parser.add_argument('--output_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
```

In [1]:
%%writefile ./src/train.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.mx.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

def train(args):
    
    # Parse arguments
    epochs = args.epochs
    pred_length = args.pred_length
    batch_size = args.batch_size
    lr = args.lr
    
    model_dir = args.model_dir
    data_dir = args.data_dir
    num_gpus = args.num_gpus
    output_dir = args.output_dir
    
    device = "gpu" if num_gpus > 0 else "cpu"
    FREQ = 'H'
    target_col = 'traffic_volume'
    related_cols = ['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description']
       
    # Get training data
    target_train_df = pd.read_csv(os.path.join(data_dir, 'target_train.csv'), index_col=0)
    related_train_df = pd.read_csv(os.path.join(data_dir, 'related_train.csv'), index_col=0)
    
    num_steps, num_series = target_train_df.shape
    target = target_train_df.values

    start_train_dt = '2017-01-01 00:00:00'
    custom_ds_metadata = {'num_series': num_series,
                          'num_steps': num_steps,
                          'prediction_length': pred_length,
                          'freq': FREQ,
                          'start': start_train_dt
                         }


    # Prepare GlounTS Dataset
    related_list = [related_train_df[c].values for c in related_cols]
    train_lst = []

    target_vec = target[:-pred_length].squeeze()
    related_vecs = [related[:-pred_length].squeeze() for related in related_list]
    dic = {FieldName.TARGET: target_vec, 
           FieldName.START: start_train_dt,
           FieldName.FEAT_DYNAMIC_REAL: related_vecs
          } 
    train_lst.append(dic)

    test_lst = []

    target_vec = target.squeeze()
    related_vecs = [related.squeeze() for related in related_list]
    dic = {FieldName.TARGET: target_vec, 
           FieldName.START: start_train_dt,
           FieldName.FEAT_DYNAMIC_REAL: related_vecs
          } 
    test_lst.append(dic)

    train_ds = ListDataset(train_lst, freq=FREQ)
    test_ds = ListDataset(test_lst, freq=FREQ)      

    # Define Estimator    
    trainer = Trainer(
        ctx=device,
        epochs=epochs,
        learning_rate=lr,
        batch_size=batch_size
    )
    
    mlp_estimator = SimpleFeedForwardEstimator(
        num_hidden_dimensions=[50],
        prediction_length=pred_length,
        context_length=2*pred_length,
        freq=FREQ,
        trainer=trainer
    )

    # Train the model
    mlp_predictor = mlp_estimator.train(train_ds)
    
    # Evaluate trained model on test data
    forecast_it, ts_it = make_evaluation_predictions(test_ds, mlp_predictor, num_samples=100)
    forecasts = list(forecast_it)
    tss = list(ts_it)
    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
    agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds))

    metrics = ['RMSE', 'MAPE', 'wQuantileLoss[0.1]', 'wQuantileLoss[0.5]', 'wQuantileLoss[0.9]', 'mean_wQuantileLoss']
    metrics_dic = dict((key,value) for key, value in agg_metrics.items() if key in metrics)
    print(json.dumps(metrics_dic, indent=2))

    # Save the model
    mlp_predictor.serialize(pathlib.Path(model_dir))
    return mlp_predictor


def parse_args():
    parser = argparse.ArgumentParser()
    
    # Hyperparameter Setting
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--pred_length', type=int, default=24*7)    
    parser.add_argument('--batch_size', type=float, default=32)
    parser.add_argument('--lr', type=float, default=0.001) 
    
    # SageMaker Container Environment
    parser.add_argument('--model_dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data_dir', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    parser.add_argument('--num_gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--output_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    
    args = parser.parse_args()
    return args    

if __name__ == '__main__':
    args = parse_args()
    train(args)    

Overwriting ./src/train.py


<br>

## 2. Training
---

스크립트가 준비되었다면 SageMaker 훈련을 수행하는 법은 매우 간단합니다. SageMaker Python SDK 활용 시, Estimator 인스턴스를 생성하고 해당 인스턴스의 `fit()` 메서드를 호출하는 것이 전부입니다. 좀 더 자세히 기술해 보면 아래와 같습니다.

#### 1) Estimator 인스턴스 생성
훈련 컨테이너에 필요한 설정들을 지정합니다. 본 핸즈온에서는 훈련 스크립트 파일이 포함된 경로인 소스 경로와(source_dir)와 훈련 스크립트 Python 파일만 엔트리포인트(entry_point)로 지정해 주면 됩니다.

#### 2) `fit()` 메서드 호출
`estimator.fit(YOUR_TRAINING_DATA_URI)` 메서드를 호출하면, 훈련에 필요한 인스턴스를 시작하고 컨테이너 환경을 시작합니다. 필수 인자값은 훈련 데이터가 존해자는 S3 경로(`s3://`)이며, 로컬 모드로 훈련 시에는 S3 경로와 로컬 경로(`file://`)를 모두 지정할 수 있습니다.

인자값 중 wait은 디폴트 값으로 `wait=True`이며, 모든 훈련 작업이 완료될 때까지 코드 셀이 freezing됩니다. 만약 다른 코드 셀을 실행하거나, 다른 훈련 job을 시작하고 싶다면 `wait=False`로 설정하여 Asynchronous 모드로 변경하면 됩니다.

**SageMaker 훈련이 끝나면 컨테이너 환경과 훈련 인스턴스는 자동으로 삭제됩니다.** 이 때, SageMaker는 자동으로 `SM_MODEL_DIR` 경로에 저장된 최종 모델 아티팩트를 `model.tar.gz`로 압축하여 훈련 컨테이너 환경에서 S3 bucket으로 저장합니다. 당연히, S3 bucket에 저장된 모델 아티팩트를 다운로드받아 로컬 상에서 곧바로 테스트할 수 있습니다.

In [2]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

### Upload data to Amazon S3

Amazon SageMaker로 모델 훈련을 실행하기 위해, 데이터를 S3에 업로드합니다. 참고로, 로컬 모드에서 테스트 시에는 S3에 업로드할 필요 없이 로컬 상에서도 훈련이 가능합니다.

In [3]:
prefix = 'timeseries-hol/traffic-volume/train'
s3_bucket = boto3.Session().resource('s3').Bucket(bucket)

s3_bucket.Object(os.path.join(prefix, 'target_train.csv')).upload_file('data/target_train.csv')
s3_bucket.Object(os.path.join(prefix, 'related_train.csv')).upload_file('data/related_train.csv')

### Local Mode Training

로컬 모드는 여러분이 작성한 훈련 및 배포 스크립트를 SageMaker에서 관리하는 클러스터에서 실행하기 전에 로컬 상(예: SageMaker 노트북 인스턴스, 개인 랩탑, 온프레미스)에서 여러분의 스크립트가 잘 동작하는지 빠르게 디버깅할 수 있는 방법입니다. 로컬 모드 훈련을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)가 필요하지만, SageMaker 노트북 인스턴스는 이미 docker가 설치되어 있습니다.

In [4]:
estimator = MXNet(entry_point='train.py',
                    source_dir='src',
                    role=role,
                    instance_type='local',
                    instance_count=1,
                    framework_version='1.6.0',
                    py_version='py3',
                    hyperparameters = {'epochs': 2, 
                                       'lr': 0.001
                                      }                       
                   )

In [5]:
s3_input = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}'.format(bucket, prefix))
estimator.fit(s3_input)

Creating skco3ilxfq-algo-1-fvoxa ... 
Creating skco3ilxfq-algo-1-fvoxa ... done
Attaching to skco3ilxfq-algo-1-fvoxa
skco3ilxfq-algo-1-fvoxa | 2021-04-07 06:21:41,691 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
skco3ilxfq-algo-1-fvoxa | 2021-04-07 06:21:41,694 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
skco3ilxfq-algo-1-fvoxa | 2021-04-07 06:21:41,708 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1-fvoxa"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"epochs":2,"lr":0.001}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1-fvoxa","hosts":["algo-1-fvoxa"]}', 'SM_INPUT_DATA_CONFIG': '{"training":{"TrainingInputMode":"File"}}', 'SM_OUTPUT_DATA_DIR': '/opt/ml/output/data', 'SM_CHANNELS': '["training"]', 'SM_CURRENT_HOST': 'algo-1-fvoxa', 'SM_MODULE_NAME': 'train', 'SM_LOG_LEVEL': '20', 'SM_FR

     |████████████████████████████████| 5.4MB 42.4MB/s 


skco3ilxfq-algo-1-fvoxa | Installing collected packages: pandas, dataclasses, pydantic, ujson, korean-lunar-calendar, hijri-converter, pymeeus, convertdate, holidays, toolz, gluonts
skco3ilxfq-algo-1-fvoxa |   Found existing installation: pandas 0.25.1
skco3ilxfq-algo-1-fvoxa |     Uninstalling pandas-0.25.1:
skco3ilxfq-algo-1-fvoxa |       Successfully uninstalled pandas-0.25.1
skco3ilxfq-algo-1-fvoxa |     Running setup.py install for ujson ... done
skco3ilxfq-algo-1-fvoxa |     Running setup.py install for pymeeus ... done
skco3ilxfq-algo-1-fvoxa | Successfully installed convertdate-2.3.2 dataclasses-0.8 gluonts-0.6.7 hijri-converter-2.1.1 holidays-0.11.1 korean-lunar-calendar-0.2.1 pandas-1.1.5 pydantic-1.6.1 pymeeus-0.5.11 toolz-0.11.1 ujson-1.35
skco3ilxfq-algo-1-fvoxa | WARNING: You are using pip version 19.3.1; however, version 21.0.1 is available.
skco3ilxfq-algo-1-fvoxa | You should consider upgrading via the 'pip install --upgrade pip' command.
skco3ilxfq-algo-1-fvoxa | 2021

100% 50/50 [00:01<00:00, 30.33it/s, epoch=2/2, avg_epoch_loss=8.96]
skco3ilxfq-algo-1-fvoxa | INFO:gluonts.trainer:Epoch[1] Elapsed time 1.649 seconds
skco3ilxfq-algo-1-fvoxa | INFO:gluonts.trainer:Epoch[1] Evaluation metric 'epoch_loss'=8.960116
skco3ilxfq-algo-1-fvoxa | INFO:root:Computing averaged parameters.
skco3ilxfq-algo-1-fvoxa | INFO:root:Loading averaged parameters.
skco3ilxfq-algo-1-fvoxa | INFO:gluonts.trainer:End model training
Running evaluation: 100% 1/1 [00:00<00:00, 27.42it/s]
skco3ilxfq-algo-1-fvoxa | {
skco3ilxfq-algo-1-fvoxa |   "MAPE": 0.5845337025603288,
skco3ilxfq-algo-1-fvoxa |   "RMSE": 2136.6897406078983,
skco3ilxfq-algo-1-fvoxa |   "wQuantileLoss[0.1]": 0.1391725132451773,
skco3ilxfq-algo-1-fvoxa |   "wQuantileLoss[0.5]": 0.34660599307192697,
skco3ilxfq-algo-1-fvoxa |   "wQuantileLoss[0.9]": 0.2493973905346963,
skco3ilxfq-algo-1-fvoxa |   "mean_wQuantileLoss": 0.24505863228393354
skco3ilxfq-algo-1-fvoxa | }
skco3ilxfq-algo-1-fvoxa | WARNING:root:Serializing R

현재 실행 중인 도커 컨테이너가 없는 것을 확인할 수 있습니다.

In [6]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


### SageMaker Hosted Training

훈련 코드가 로컬에서 잘 작동하므로, 이제 SageMaker에서 관리하는 훈련 인스턴스를 사용하여 훈련을 수행하겠습니다. 로컬 모드 훈련과 달리 호스팅 훈련은
노트북 인스턴스 대신에 SageMaker에서 관리하는 별도의 클러스터에서 수행합니다. 본 핸즈온의 데이터셋 사이즈가 작기 때문에 체감이 되지 않겠지만, 대규모 데이터 및 복잡한 모델에 대한 분산 훈련은 SageMaker 호스팅 훈련 방법을 사용하는 것을 권장합니다.

In [7]:
estimator = MXNet(entry_point='train.py',
                    source_dir='src',
                    role=role,
                    train_instance_type='ml.c5.xlarge',
                    train_instance_count=1,
                    framework_version='1.6.0',
                    py_version='py3',
                    hyperparameters = {'epochs': 20, 
                                       'lr': 0.001,
                                      }                       
                   )

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [8]:
s3_input = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}'.format(bucket, prefix))
estimator.fit(s3_input)

2021-04-07 06:22:04 Starting - Starting the training job...
2021-04-07 06:22:27 Starting - Launching requested ML instancesProfilerReport-1617776524: InProgress
.........
2021-04-07 06:23:47 Starting - Preparing the instances for training...
2021-04-07 06:24:27 Downloading - Downloading input data...
2021-04-07 06:24:51 Training - Training image download completed. Training in progress.2021-04-07 06:24:51,649 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
2021-04-07 06:24:51,651 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-04-07 06:24:51,662 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"epochs":20,"lr":0.001}', 'SM_USER_ENTRY_POINT': 'train.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
[2021-04-07 06:25:04.772 ip-10-2-254-154.ec2.internal:53 INFO json_config.py:90] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-04-07 06:25:04.773 ip-10-2-254-154.ec2.internal:53 INFO hook.py:193] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-04-07 06:25:04.773 ip-10-2-254-154.ec2.internal:53 INFO hook.py:238] Saving to /opt/ml/output/tensors
[2021-04-07 06:25:04.773 ip-10-2-254-154.ec2.internal:53 INFO state_store.py:67] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-04-07 06:25:04.795 ip-10-2-254-154.ec2.internal:53 INFO hook.py:398] Monitoring the collections: losses
[2021-04-07 06:25:04.795 ip-10-2-254-154.ec2.internal:53 INFO hook.py:461] Hook is writing from the hook with pid: 53

#015100%|██████████| 50/50 [00:02<00:00, 19.51it/s, epoch=1/20, avg_epo


2021-04-07 06:25:51 Uploading - Uploading generated training model
2021-04-07 06:25:51 Completed - Training job completed
Training seconds: 92
Billable seconds: 92


<br>

## 3. Getting Model Artifacts
---

훈련이 완료된 모델 아티팩트를 로컬(노트북 인스턴스 or 온프레미스)로 복사합니다. 훈련 완료 시 `SM_MODEL_DIR`에 있는 파일들이
`model.tar.gz`로 자동으로 압축되며, 압축을 해제하여 로컬 상에서도 추론을 수행할 수 있습니다.

In [9]:
local_model_dir = './model'
!rm -rf $local_model_dir

In [10]:
import json , os

s3_model_dir = estimator.model_data.replace('model.tar.gz', '')
print(s3_model_dir)
!aws s3 ls {s3_model_dir}

if not os.path.exists(local_model_dir):
    os.makedirs(local_model_dir)

!aws s3 cp {s3_model_dir}model.tar.gz {local_model_dir}/model.tar.gz
!tar -xzf {local_model_dir}/model.tar.gz -C {local_model_dir}

s3://sagemaker-us-east-1-143656149352/mxnet-training-2021-04-07-06-22-03-922/output/
2021-04-07 06:25:47   10443872 model.tar.gz
download: s3://sagemaker-us-east-1-143656149352/mxnet-training-2021-04-07-06-22-03-922/output/model.tar.gz to model/model.tar.gz


다음 모듈에서 활용할 변수들을 저장합니다.

In [11]:
%store s3_model_dir
%store prefix

Stored 's3_model_dir' (str)
Stored 'prefix' (str)
